In [87]:
# imports

from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from pypdf import PdfReader
import gradio as gr

In [88]:
# The usual start

load_dotenv(override=True)
openai = OpenAI()
client = OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama"  # dummy key
)

In [89]:
TELEGRAM_TOKEN = os.getenv("TELEGRAM_BOT_TOKEN")
TELEGRAM_CHAT_ID = os.getenv("TELEGRAM_CHAT_ID")
telegram_url = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendMessage"


In [90]:

def notify(message):
    base_url = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}"
    url = f"{base_url}/sendMessage"
    
    text = f"🤖 **CAREER CHATBOT LOG** 🤖\n\n{message}"
    
    # 💡 ADD A VARIABLE TO CAPTURE THE RESPONSE
    response = requests.post(url, data={'chat_id': TELEGRAM_CHAT_ID, 'text': text, 'parse_mode': 'Markdown'})
    
    # 💡 ADD PRINT STATEMENTS TO LOG ERRORS
    if response.status_code != 200:
        print(f"--- TELEGRAM NOTIFICATION ERROR ---")
        print(f"Status Code: {response.status_code}")
        print(f"Response: {response.text}")
        print(f"-----------------------------------")
    
    return response.status_code

In [91]:
def record_user_details(email, name="Name not provided", notes="not provided"):
    notify(f"New visitor ➜ {name}, Email: {email}, Notes: {notes}")
    return {"recorded": "ok"}

def record_unknown_question(question):
    notify(f"Unknown question logged ➜ {question}")
    return {"recorded": "ok"}

In [92]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Record when a user provides an email",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {"type": "string"},
            "name": {"type": "string"},
            "notes": {"type": "string"}
        },
        "required": ["email"]
    }
}


In [93]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Record an unanswered question",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {"type": "string"}
        },
        "required": ["question"]
    }
}

In [94]:
tools = [
    {"type": "function", "function": record_user_details_json},
    {"type": "function", "function": record_unknown_question_json}
]

In [95]:
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        args = json.loads(tool_call.function.arguments)
        tool_fn = globals().get(tool_name)
        result = tool_fn(**args)
        results.append({
            "role": "tool",
            "content": json.dumps(result),
            "tool_call_id": tool_call.id
        })
    return results


In [96]:
reader = PdfReader("me/Annabatula Sai Harsha CV Simple (1).pdf")
linkedin = "".join([page.extract_text() or "" for page in reader.pages])

with open("me/new_summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

name = "A.S Harsha"  # ← Change for yourself

In [97]:
system_prompt = f"""
You are acting as {name}. Answer questions about {name}'s career, education, experience, projects, 
and background using the summary and LinkedIn profile below.

If you cannot answer something → call record_unknown_question.
If the user shows interest → ask for email and call record_user_details.

Be professional, friendly, and helpful.

## Summary:
{summary}

## LinkedIn:
{linkedin}

Stay in character as {name} throughout.
"""



In [98]:
def chat(message, history):

    messages = [{"role": "system", "content": system_prompt}] \
                + history \
                + [{"role": "user", "content": message}]

    done = False
    while not done:
        response = client.chat.completions.create(
            model="llama3.2",     # ← Your Llama model
            messages=messages,
            tools=tools
        )

        finish = response.choices[0].finish_reason

        if finish == "tool_calls":
            tool_calls = response.choices[0].message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(response.choices[0].message)
            messages.extend(results)
        else:
            done = True

    return response.choices[0].message.content



In [99]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.
